In [ ]:
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network._multilayer_perceptron import MLPRegressor
from statsmodels.tsa.statespace.sarimax import SARIMAX
from google.oauth2 import service_account
from datetime import datetime as date
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.metrics import r2_score,make_scorer,mean_squared_error
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_regression
from statsmodels.tsa.stattools import adfuller

import joblib
import pickle
import pandas as pd
import pandas_gbq
import numpy as np
import xgboost
import lightgbm
import os

### Gathering Data and Feature Engineering

In [ ]:
player_features= ["min_3gm_avg", "fgm_3gm_avg", "fga_3gm_avg", "fg%_3gm_avg", "3pm_3gm_avg", 
                   "3pa_3gm_avg", "3p%_3gm_avg", "ftm_3gm_avg", "fta_3gm_avg", "ft%_3gm_avg", 
                   "oreb_3gm_avg", "dreb_3gm_avg", "reb_3gm_avg", "ast_3gm_avg", "stl_3gm_avg", 
                   "blk_3gm_avg", "to_3gm_avg", "pf_3gm_avg", "pts_3gm_avg", "plus_mins_3gm_avg",
                   "pts_season", "pts_momentum", "min_season", "min_momentum", "fgm_season", "fgm_momentum", 
                    "fga_season", "fga_momentum", "fg%_season", "fg%_momentum", "3pm_season", "3pm_momentum", 
                    "3pa_season", "3pa_momentum", "3p%_season", "3p%_momentum", "ftm_season", "ftm_momentum", 
                    "fta_season", "fta_momentum", "ft%_season", "ft%_momentum", "oreb_season", "oreb_momentum", 
                    "dreb_season", "dreb_momentum", "reb_season", "reb_momentum", "ast_season", "ast_momentum", 
                    "stl_season", "stl_momentum", "blk_season", "blk_momentum", "to_season", "to_momentum", 
                    "pf_season", "pf_momentum", "plus_mins_season", "plus_mins_momentum"]

team_features = ["home","away","offrtg_3gm_avg", "defrtg_3gm_avg", "netrtg_3gm_avg", "ast%_3gm_avg", "ast_to_3gm_avg", 
                    "ast_ratio_3gm_avg", "oreb%_3gm_avg", "dreb%_3gm_avg", "reb%_3gm_avg", "tov%_3gm_avg", 
                    "efg%_3gm_avg", "ts%_3gm_avg", "pace_3gm_avg", "pie_3gm_avg",
                    "netrtg_season", "netrtg_momentum", "offrtg_season", "offrtg_momentum", "defrtg_season", "defrtg_momentum", 
                    "ast%_season", "ast%_momentum", "ast_to_season", "ast_to_momentum", "ast_ratio_season", "ast_ratio_momentum", 
                    "oreb%_season", "oreb%_momentum", "dreb%_season", "dreb%_momentum", "reb%_season", "reb%_momentum", 
                    "tov%_season", "tov%_momentum", "efg%_season", "efg%_momentum", "ts%_season", "ts%_momentum", 
                    "pace_season", "pace_momentum", "pie_season", "pie_momentum"]

#using shifted windows for rolling data to prevent data leakage
player_query = f""" 
SELECT player,team,game_id,game_date,matchup,pts,reb,ast,`3pm`, {','.join([f'`{player}`' for player in player_features])},season
from `capstone_data.player_modeling_data`
order by game_date asc
"""

team_query = f"""
SELECT team,game_id,game_date,home,away, {', '.join([f'`{team}`' for team in team_features])}
from `capstone_data.team_modeling_data`
order by game_date asc
"""


In [ ]:
try:
    full_data = pd.read_csv('full_data.csv')

except:
    nba_player_data = pd.DataFrame(pandas_gbq.read_gbq(player_query,project_id='miscellaneous-projects-444203'))
    team_data = pd.DataFrame(pandas_gbq.read_gbq(team_query,project_id='miscellaneous-projects-444203'))
    opponent_data = team_data.rename(columns={
    col: ('matchup' if col == 'team' else 'game_id' if col == 'game_id' else f'opponent_{col}') for col in team_data.columns})
    full_data = nba_player_data.merge(team_data, on = ['game_id','team'], how = 'inner',suffixes=('','remove'))
    full_data = full_data.merge(opponent_data,on = ['game_id','matchup'],how = 'inner',suffixes=('','remove'))
    full_data.drop([column for column in full_data.columns if 'remove' in column],axis = 1 , inplace=True) 
    full_data.drop([column for column in full_data.columns if '_1' in column],axis = 1 , inplace=True)
    full_data.to_csv('full_data.csv',mode = 'x')

In [ ]:
data_ordered = full_data.sort_values('game_date')

data_ordered.dropna(inplace=True)


#### Feature Engineering Ideas 

* (ratio of 3pa and fga and 3pm and 3pa) TS% for players efg% 
* for players assist_to_turnover ratio assist ratio, 
* rebound_cahnce, defesnive reb %, 
* ast_ratio_season * pace, 
* home * pts season - data pts 3pm avg,
* cold_streak pts_3gm_avg < pts_season boolean, 
* away difficulty away * opponent_defrtg_3gm_avg,
* home_performance = data_ordered[data_ordered["home"] == 1].groupby("team")["pts_season"].mean()
* away_performance = data_ordered[data_ordered["away"] == 1].groupby("team")["pts_season"].mean() these would be to see how the team performance changes 


In [ ]:
data_ordered['pts_per_min_3gm'] = data_ordered['pts_3gm_avg']/data_ordered['min_3gm_avg']
data_ordered['pts_per_min_season'] = data_ordered['pts_season']/data_ordered['min_season']
data_ordered['pts_per_min_momentum'] = data_ordered['pts_per_min_3gm'] - data_ordered['pts_per_min_season']

data_ordered['3pm_per_min_3gm'] = data_ordered['3pm_3gm_avg']/data_ordered['min_3gm_avg']
data_ordered['3pm_per_min_season'] = data_ordered['3pm_season']/data_ordered['min_season']
data_ordered['3pm_per_min_momentum'] = data_ordered['3pm_per_min_3gm'] - data_ordered['3pm_per_min_season'] 

data_ordered['reb_per_min_3gm'] = data_ordered['reb_3gm_avg']/data_ordered['min_3gm_avg']
data_ordered['reb_per_min_season'] = data_ordered['reb_season']/data_ordered['min_season']
data_ordered['reb_per_min_momentum'] = data_ordered['3pm_per_min_3gm'] - data_ordered['reb_per_min_season']

home_performance = data_ordered[data_ordered['home'] == 1]
away_performance = data_ordered[data_ordered['away'] == 1]

In [ ]:
# # Ensure data is sorted correctly for chronological calculations
# data_ordered = data_ordered.sort_values(by=['player', 'season', 'game_date'])

# # Separate home and away games
# home_performance = data_ordered[data_ordered['home'] == 1]
# away_performance = data_ordered[data_ordered['home'] == 0]  # Fixed to align with `home` flag

# # Compute season-to-date averages for home and away games (including game_id)
# home_rolling = (
#     home_performance.groupby(['player', 'season'])[['game_id', 'pts', 'reb', 'ast', '3pm']]
#     .apply(lambda x: x.set_index('game_id').expanding().mean().shift(1))  # Prevent data leakage
#     .reset_index()
# )

# away_rolling = (
#     away_performance.groupby(['player', 'season'])[['game_id', 'pts', 'reb', 'ast', '3pm']]
#     .apply(lambda x: x.set_index('game_id').expanding().mean().shift(1))
#     .reset_index()
# )

# # Rename columns before merging
# home_rolling = home_rolling.rename(columns={'pts': 'home_avg_pts', 'reb': 'home_avg_reb', 
#                                             'ast': 'home_avg_ast', '3pm': 'home_avg_3pm'})
# away_rolling = away_rolling.rename(columns={'pts': 'away_avg_pts', 'reb': 'away_avg_reb', 
#                                             'ast': 'away_avg_ast', '3pm': 'away_avg_3pm'})

# # Merge rolling averages back into `data_ordered`
# data_ordered = data_ordered.merge(home_rolling[['player', 'game_id', 'home_avg_pts', 'home_avg_reb', 'home_avg_ast', 'home_avg_3pm']],
#                                   on=['player', 'game_id'], how='left')

# data_ordered = data_ordered.merge(away_rolling[['player', 'game_id', 'away_avg_pts', 'away_avg_reb', 'away_avg_ast', 'away_avg_3pm']],
#                                   on=['player', 'game_id'], how='left')

# # Fill missing values for early season games
# for cat in ['pts', 'reb', 'ast', '3pm']:
#     data_ordered[f'home_avg_{cat}'] = data_ordered[f'home_avg_{cat}'].fillna(0)
#     data_ordered[f'away_avg_{cat}'] = data_ordered[f'away_avg_{cat}'].fillna(0)

#     # Compute home vs. away performance difference conditionally
#     data_ordered[f'{cat}_home_away_diff'] = (
#         (data_ordered['home'] == 1) * (data_ordered[f'home_avg_{cat}'] - data_ordered[f'away_avg_{cat}']) +
#         (data_ordered['home'] == 0) * (data_ordered[f'away_avg_{cat}'] - data_ordered[f'home_avg_{cat}'])
#     )

# # Drop unnecessary columns
# data_ordered = data_ordered.drop(columns=[f'home_avg_{cat}' for cat in ['pts', 'reb', 'ast', '3pm']] + 
#                                            [f'away_avg_{cat}' for cat in ['pts', 'reb', 'ast', '3pm']])



In [ ]:
data_ordered = data_ordered.groupby(['player','season']).apply(lambda x: x.iloc[3:]).reset_index(drop=True)

In [ ]:
data_ordered.sort_values(by='game_date',inplace=True)

In [ ]:
data_ordered['game_date'] = pd.to_datetime(data_ordered['game_date'])

In [ ]:
data_ordered['days_ago'] = (data_ordered['game_date'].max() - data_ordered['game_date']).dt.days
data_ordered['time_decay_weight'] = 1 / (1 + np.log(1 + data_ordered['days_ago']))

In [ ]:
pd.set_option('display.max_columns',100000)

In [ ]:
data_ordered = data_ordered.drop('Unnamed: 0', axis =1)

In [ ]:
players = data_ordered[data_ordered['season']=='2024-2025']['player'].unique()

In [ ]:
numeric_columns = data_ordered.select_dtypes(include=['number']).columns.tolist()
numeric_columns = [column for column in numeric_columns if column not in ['pts','reb','ast','blk','stl','3pm','game_id','game_date','days_ago','time_decay_weight']]

player_features = {player:{feature:[] for feature in ['pts','reb','ast','3pm']}for player in players}

In [ ]:

for player in player_features.keys():
    for category in player_features[player].keys():
        player_data = data_ordered[data_ordered['player'] == player]

        if len(player_data) > 82:  # Use player-specific data
            x = player_data[numeric_columns]
            y = player_data[category]
        else:  # Use global data
            x = data_ordered[numeric_columns]
            y = data_ordered[category]

        mi_scores = mutual_info_regression(x, y)
        mi_scores = pd.Series(mi_scores, index=numeric_columns)
        selected_features = mi_scores[mi_scores > 0.05].index.tolist()  # Keep features with MI > 0.05

        player_features[player][category] = selected_features


In [ ]:
for player in player_features.keys():
    for category in player_features[player].keys():
        print(category)
        for column in numeric_columns:
                player_data = data_ordered[data_ordered['player'] == player]
                if len(player_data) > 82 : 
                    pearson_corr, pearson_p = pearsonr(player_data[column], player_data[category])
                    spearman_corr, spearman_p = spearmanr(player_data[column], player_data[category])
                    
                    if abs(spearman_corr) > 0.3 and abs(pearson_corr) < 0.2:
                        print(f"🚀 {column} likely has a non-linear relationship with {category}")
                else: 
                    pearson_corr, pearson_p = pearsonr(data_ordered[column], data_ordered[category])
                    spearman_corr, spearman_p = spearmanr(data_ordered[column], data_ordered[category])

                    if abs(spearman_corr) > 0.3 and abs(pearson_corr) < 0.2:
                        print(f"🚀 {column} likely has a non-linear relationship with {category}")
                # If Spearman is high but Pearson is low, it suggests a non-linear relationship


In [ ]:
#These values appeared to have non-linear relationships applying transformations
data_ordered['ft%_season'] = np.log1p(data_ordered['ft%_season'])
data_ordered['stl_3gm_avg'] = np.log1p(data_ordered['stl_3gm_avg'])
data_ordered['stl_season'] = np.log1p(data_ordered['stl_season'])
data_ordered['to_season'] = data_ordered['to_season']**2 
data_ordered['to_3gm_avg'] = data_ordered['to_3gm_avg']**2 

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
saved_models = {player:{category:{} for category in ['pts','reb','ast','3pm']} for player in players}
saved_results = {player:{category:{} for category in ['pts','reb','ast','3pm']}for player in players}

In [ ]:
for cat in player_features['Damian Lillard']['ast']:
    print(cat)

#### SHAP
Applying shap to help reduce collinearity

### Linear Model

In [ ]:
for player in player_features.keys():
    for category in player_features[player].keys():
        player_features[player][category] = [f for f in player_features[player][category] if f != category]


In [ ]:

for player in player_features.keys():
    player_data = data_ordered[data_ordered['player'] == player]
    if len(player_data) > 82:
        print('enough data for player sepcific model')
        player_data = player_data.sort_values(by='game_date')
        split_index = int(len(player_data) * .80)

        train_data = player_data.iloc[:split_index]
        test_data = player_data[split_index:]

    else:
        split_index = int(len(data_ordered) * .80)

        train_data = data_ordered.iloc[:split_index]
        test_data = data_ordered[split_index:]

    for category in player_features[player].keys():
        if len(player_features[player][category]) == 0:
            continue
        features_list = [f for f in player_features[player][category] if f != category]
        print(len(features_list))
        x_train,y_train = train_data[features_list],train_data[category]
        x_test, y_test = test_data[features_list],test_data[category]
        linear_model = LinearRegression()

        linear_model.fit(x_train,y_train)

        y_pred = linear_model.predict(x_test)
        print(player,category)
        print(r2_score(y_true=y_test,y_pred=y_pred))

        saved_results[player][category]['linear_model']={'r2':{r2_score(y_true=y_test,y_pred=y_pred)}, 'mse':{mean_squared_error(y_true=y_test,y_pred=y_pred)}}
        saved_models[player][category]['linear_model'] = linear_model

In [ ]:
#Assessing for multicollinearity
mulitcol_pairs = {cat:[] for cat in features.keys()}
for category in features.keys():
    corr_matrix = data_ordered[features[category]].corr()
    high_corr_vars = np.where(abs(corr_matrix) > 0.8)
    high_corr_pairs = [(corr_matrix.index[x], corr_matrix.columns[y]) 
                    for x, y in zip(*high_corr_vars) if x != y and x < y]
    mulitcol_pairs[category].append(high_corr_pairs)
      # Drop one from each highly correlated pair


In [ ]:
trimmed_feats = {cat:[] for cat in features.keys()}
for cat in mulitcol_pairs.keys():
    print(cat)
    for pairs in mulitcol_pairs[cat]:
        for x,y in pairs:
            cor_x = pearsonr(data_ordered[x],data_ordered[cat])
            cor_y = pearsonr(data_ordered[y],data_ordered[cat])

            if cor_x[1] < .05 and cor_x[0] > cor_y[0]:
                trimmed_feats[cat].append(x)
            elif cor_y[1] < .05 and cor_y[0] > cor_x[0]:
                trimmed_feats[cat].append(y)

In [ ]:
for category in features.keys():
    features_list = [f for f in features[category] if f != category]
    x_train,y_train = train_data[features_list],train_data[category]
    x_test, y_test = test_data[features_list],test_data[category]
    ridge_model = Ridge(alpha=1)

    ridge_model.fit(x_train,y_train)

    output = pd.DataFrame({'prediction':ridge_model.predict(x_test), 'actual':y_test})
    print(category)
    print(r2_score(y_true=output['actual'],y_pred=output['prediction']))

### XGboost

In [ ]:
scaler = StandardScaler()

scaled_data = scaler.fit_transform(data_ordered[numeric_columns])

scaled_data_df = pd.DataFrame(scaled_data,columns=numeric_columns)

split_index = int(len(data_ordered) * .80)

scaled_train_data = scaled_data_df.iloc[:split_index]
scaled_test_data = scaled_data_df[split_index:]

In [ ]:
param_grid = {'max_depth':[3,6,9],'learning_rate':[.01,.05,.1,.3],'booster':['gbtree','dart'],'subsample':[.5,.7,.9],'colsample_bytree':[.5,.7,.9],'n_estimators':[100,300,500]}
param_linear = {'booster':['gblinear'],'lambda':[0,.1,1,10],'alpha':[0,.1,1,10]}

In [ ]:
xgb_regressor = xgboost.XGBRegressor()
mse_score = make_scorer(mean_squared_error,greater_is_better=False)
r2_scorer = make_scorer(r2_score)
scoring = {'MSE':mse_score,'r2':r2_scorer}
grid_search = GridSearchCV(estimator=xgb_regressor,param_grid=param_grid,scoring = scoring,cv=tscv,n_jobs=1,verbose=0,refit='r2')
grid_linear_search = GridSearchCV(estimator=xgb_regressor,param_grid=param_linear,scoring = scoring,cv=tscv,n_jobs=3,verbose=0,refit='r2')


In [ ]:
xg_features = [feature for feature in data_ordered.columns if data_ordered[feature].dtype == 'float' and feature not in features.keys() and feature != 'time_decay_weight']

In [ ]:
for category in features.keys():
    x_train,y_train = scaled_train_data[xg_features],train_data[category]
    x_test, y_test = scaled_test_data[xg_features],test_data[category]

    fit_params = {'eval_set':[(x_test,y_test)],'early_stopping_rounds':20,'verbose':False}

    grid_linear_search.estimator.set_params(eval_metric='rmse')


    grid_linear_search.fit(x_train,y_train)


    print(category)
    print(grid_linear_search.best_params_)
    print(grid_linear_search.best_score_)

    y_pred = grid_linear_search.best_estimator_.predict(x_test)

    saved_models[category]['XGboost'] = grid_linear_search.best_estimator_
    saved_results[category]['XGboost']={'r2':{r2_score(y_true=y_test,y_pred=y_pred)}, 'mse':{mean_squared_error(y_true=y_test,y_pred=y_pred)}}


### LightGBM

In [ ]:
light = lightgbm.LGBMRegressor(boosting_type='gbdt', n_estimators=500)  # Using hist for faster training

param_grid = {
    'num_leaves': [31, 50],  
    'learning_rate': [0.01, 0.1],  
    'max_depth': [-1, 10],  
}


In [ ]:
light_grid_search = GridSearchCV(estimator=light,param_grid=param_grid,cv=tscv,verbose=0,n_jobs=4)

In [ ]:
split_index = int(len(data_ordered) * .80)

train_data = data_ordered.iloc[:split_index]
test_data = data_ordered[split_index:]
for category in features.keys():
    x_train,y_train = train_data[features[category]],train_data[category]
    x_test,y_test = test_data[features[category]],test_data[category]

    light_grid_search.fit(x_train,y_train)

    best_model = light_grid_search.best_estimator_
    print(category)
    print("Best Parameters:", light_grid_search.best_params_)

    y_pred = best_model.predict(x_test)

    mse = mean_squared_error(y_test,y_pred)
    r2 = r2_score(y_test,y_pred)

    saved_models[category]['lightgbm'] = best_model
    print(f'MSE: {mse}')
    print(f'R2: {r2}')

    saved_results[category]['lightgbm']={'r2':{r2_score(y_true=y_test,y_pred=y_pred)}, 'mse':{mean_squared_error(y_true=y_test,y_pred=y_pred)}}


In [ ]:
joblib.dump(saved_models,'models.pkl')

In [ ]:
with open('saved_performance.txt', 'w') as file:
    for category, models in saved_results.items():
        file.write(f"Category: {category}\n")
        for model, metrics in models.items():
            file.write(f"  Model: {model}\n")
            for metric, value in metrics.items():
                file.write(f"    {metric}: {value}\n")
        file.write("\n")  # Newline between categories
